### Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары

In [1]:
pip install selenium


  Using cached selenium-4.1.0-py3-none-any.whl (958 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached trio-0.19.0-py3-none-any.whl (356 kB)
  Using cached outcome-1.1.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.0.0-py3-none-any.whl (24 kB)
  Using cached h11-0.12.0-py3-none-any.whl (54 kB)


#### загружаем нужные библиотеки

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
import pandas as pd

In [5]:
client = MongoClient('localhost', 27017)
db = client['selenium']
collection = db['mvideos']

In [7]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path='chromedriver.exe', options=chrome_options)
driver.implicitly_wait(10)
driver.get('https://www.mvideo.ru/')
wait = WebDriverWait(driver, 2)

<ipython-input-7-97043fda55d1>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe', options=chrome_options)


In [8]:
while True:
    try:
        element = wait.until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(),'В тренде')]"))
        )
        element.click()
        break
    except:
        actions = ActionChains(driver)
        actions.key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN)
        actions.perform()

In [9]:
while True:
    try:

        next = wait.until(EC.element_to_be_clickable((By.XPATH, "//mvid-shelf-group/*/div[2]/button[2]")))
        next.click()
    except:
        break

In [10]:
elements = driver.find_elements(By.XPATH, '//mvid-shelf-group//a[@class="img-with-badge"]')
links = [i.get_attribute('href') for i in elements]
for i in links:
    position = {}
    driver.get(i)
    code = driver.find_element(By.CLASS_NAME, 'product-code-container').text
    name = driver.find_element(By.XPATH, '//h1[@class="title"]').text
    cost = driver.find_element(By.CLASS_NAME, 'price__main-value').text
    
    position["Код Товара"] = code
    position["Ссылка"] = i
    position["Цена"] = cost
    position["Название"] = name
    
    collection.update_one({'Ссылка': position['Ссылка']}, {'$set' : position}, upsert=True)

In [11]:
df = pd.DataFrame([i for i in collection.find({})])

In [14]:
df.head(10)

,_id,Ссылка,Код Товара,Название,Цена
0,61b2006eb8d1ab28a80adbfb,https://www.mvideo.ru/products/noutbuk-acer-sw...,Код товара: 3005 7260,Ноутбук Acer Swift 1 SF114-34-C564 NX.A74ER.002,30 999 ₽
1,61b2006fb8d1ab28a80adbfd,https://www.mvideo.ru/products/smartfon-huawei...,Код товара: 3005 3815,Смартфон Huawei P Smart 2021 4+128GB Crush Gre...,30 999 ₽
2,61b20071b8d1ab28a80adc00,https://www.mvideo.ru/products/naushniki-true-...,Код товара: 5014 1149,Наушники True Wireless Samsung Galaxy Buds Liv...,9 999 ₽
3,61b20073b8d1ab28a80adc02,https://www.mvideo.ru/products/podpiska-mprime...,Код товара: 6016151,Подписка M.Prime на 1 месяц + Яндекс.Плюс,499 ₽
4,61b20074b8d1ab28a80adc04,https://www.mvideo.ru/products/konvektor-hyund...,Код товара: 2005 5370,Конвектор Hyundai H-HV21-15-UI662,499 ₽
5,61b20076b8d1ab28a80adc07,https://www.mvideo.ru/products/smart-chasy-sam...,Код товара: 3005 1392,Смарт-часы Samsung Galaxy Watch3 45mm Черные (...,25 499 ₽
6,61b20078b8d1ab28a80adc09,https://www.mvideo.ru/products/elektrogril-tef...,Код товара: 2003 6709,Электрогриль Tefal Supergrill GC450B32,25 499 ₽
7,61b2007ab8d1ab28a80adc0b,https://www.mvideo.ru/products/elektrochainik-...,Код товара: 2007 4031,Электрочайник Tefal Sense KO693110,9 999 ₽
8,61b2007bb8d1ab28a80adc0e,https://www.mvideo.ru/products/kofemashina-kap...,Код товара: 2007 0575,Кофемашина капсульного типа Nespresso Vertuo N...,3 999 ₽
9,61b2007db8d1ab28a80adc10,https://www.mvideo.ru/products/ruchnoi-otpariv...,Код товара: 2007 2378,Ручной отпариватель Philips STH3020/10,6 999 ₽
